In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import requests

def save_html_from_dict(urls_and_filenames):
    retrieved_files = []
    for pair in urls_and_filenames:
        url = pair['url']
        file_name = pair['file_name']

        response = requests.get(url)

        if response.status_code == 200:
            html_content = response.text

            with open(file_name, "w", encoding="utf-8") as file:
                file.write(html_content)

            retrieved_files.append(file_name)
            print("HTML content from", url, "saved to", file_name)
        else:
            print("Failed to fetch content from", url)

    return retrieved_files

# URLs and corresponding file names in a list of dictionaries
urls_and_filenames = [
    {'url': 'HERE_GOES_WEBSITE_URL', 'file_name': 'output1.html'}, # There can be as many websites as needed
]

# Save HTML content from multiple URLs using the dictionary list
retrieved_files = save_html_from_dict(urls_and_filenames)

print("Retrieved files:", retrieved_files)


In [ ]:

from langchain_community.document_loaders import BSHTMLLoader

loaded_data = []

for path in retrieved_files:
    loader = BSHTMLLoader(file_path=path, open_encoding='utf-8')
    data = loader.load()
    loaded_data.extend(data)
loaded_data


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200)

docs = text_splitter.split_documents(loaded_data)
docs

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())
db

In [ ]:
docs_similarity = db.similarity_search(query='', k=4)
docs_page_content = " " .join([doc.page_content for doc in docs_similarity])

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

prompt = PromptTemplate(
    input_variables=["question", "docs"],
    template="""
    Jesteś pomocnym asystentem, który potrafi odpowiadać na pytania dotyczące zadań, wymagań i mile widzianych umiejętności w ofertach pracy na stronach internetowych.

    Odpowiedz na następujące pytanie: {question}
    Szukając następującej strony internetowej: {docs}

    Wykorzystaj tylko faktyczne informacje z transkryptu, aby odpowiedzieć na pytanie.

    Jeśli uważasz, że nie masz wystarczających informacji, aby odpowiedzieć na pytanie, powiedz "Nie wiem".

    Twoje odpowiedzi powinny być obszerne i szczegółowe.
    """,
)

chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
response = chain.run(question="Jakie są wymagania dla PHP web developera", docs=docs_page_content)
print(response)